In [1]:
import pandas as pd
import getEPH
import statsmodels.formula.api as slm
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from scipy import stats
from pandas.stats.api import ols
from sklearn import linear_model

In [40]:
#getEPH.getEPHdbf('t310')

In [164]:
data = pd.read_csv('data/cleanDataHouseholdt310.csv')
data.head()

,CODUSU,NRO_HOGAR,REGION,PONDERA,HomeType,HomeTypeesp,RoomsNumber,FloorMaterial,FloorMaterialesp,RoofMaterial,...,HouseMembers,Memberless10,Membermore10,TotalHouseHoldIncome,DomesticService1,DomesticService2,DomesticService3,DomesticService4,DomesticService5,DomesticService6
0,302468,1,1,1287,2,NaN,2,1,NaN,9,...,2,0,2,4000.0,1,0,2,0,0,0
1,307861,1,1,1674,2,NaN,2,1,NaN,1,...,4,1,3,5800.0,1,2,98,0,0,0
2,308762,1,1,1522,2,NaN,4,1,NaN,9,...,1,0,1,3200.0,1,0,98,0,0,0
3,308278,1,1,1320,2,NaN,3,1,NaN,9,...,2,0,2,10000.0,2,0,96,0,0,0
4,311937,1,1,1281,2,NaN,4,1,NaN,1,...,4,0,4,11000.0,2,4,96,0,0,0


In [165]:
data = data.query('REGION == 1')
data = data.dropna(axis = 1)
data["id"] = data.CODUSU.map(str) + data.NRO_HOGAR.map(str)
data = data.drop(['CODUSU','NRO_HOGAR', 'DomesticService1','DomesticService2',
                  'DomesticService3','DomesticService4','DomesticService5','DomesticService6'],axis = 1)
def remove9(df,variables):
    for var in variables:
        df[var].replace(to_replace=[9], value=[np.nan] , inplace=True, axis=None)

def remove0(df,variables):
    for var in variables:
        df[var].replace(to_replace=[0], value=[np.nan] , inplace=True, axis=None)

def remove99(df,variables):
    for var in variables:
        df[var].replace(to_replace=[99], value=[np.nan] , inplace=True, axis=None)

        
remove9(df = data, variables = ['FloorMaterial','RoofMaterial','RoofCoat','Water','WaterType','Toilet','ToiletLocation',
                               'ToiletType','Sewer','DumpSites','Flooding','EmergencyLoc','CookingCombustible',
                               'BathroomUse'])
remove0(df = data, variables = ['FloorMaterial','RoofMaterial','RoofCoat','Water','WaterType','Toilet','ToiletLocation',
                               'ToiletType','Sewer','DumpSites','Flooding','EmergencyLoc','Ownership','CookingCombustible',
                               'BathroomUse', 'TotalHouseHoldIncome'])
remove99(df = data, variables = ['Ownership', 'RoomsNumber'])

data.head()

,REGION,PONDERA,HomeType,RoomsNumber,FloorMaterial,RoofMaterial,RoofCoat,Water,WaterType,Toilet,...,Garage,Ownership,CookingCombustible,BathroomUse,Working,HouseMembers,Memberless10,Membermore10,TotalHouseHoldIncome,id
0,1,1287,2,2.0,1.0,NaN,1.0,1.0,1,1,...,2,3.0,1.0,1.0,2,2,0,2,4000.0,3024681
1,1,1674,2,2.0,1.0,1.0,1.0,1.0,1,1,...,2,5.0,1.0,1.0,1,4,1,3,5800.0,3078611
2,1,1522,2,4.0,1.0,NaN,1.0,1.0,1,1,...,2,1.0,1.0,1.0,2,1,0,1,3200.0,3087621
3,1,1320,2,3.0,1.0,NaN,1.0,1.0,1,1,...,2,1.0,4.0,1.0,1,2,0,2,10000.0,3082781
4,1,1281,2,4.0,1.0,1.0,1.0,1.0,1,1,...,2,1.0,1.0,1.0,1,4,0,4,11000.0,3119371


In [166]:
cols = data.columns.tolist()
cols = cols[-1:] + cols[:-1]

In [167]:
#data['weights'] = ( 1.0 / data.PONDERA )
df = data[cols]
df.TotalHouseHoldIncome = np.log(df.TotalHouseHoldIncome)
df.head()

,id,REGION,PONDERA,HomeType,RoomsNumber,FloorMaterial,RoofMaterial,RoofCoat,Water,WaterType,...,Sink,Garage,Ownership,CookingCombustible,BathroomUse,Working,HouseMembers,Memberless10,Membermore10,TotalHouseHoldIncome
0,3024681,1,1287,2,2.0,1.0,NaN,1.0,1.0,1,...,2,2,3.0,1.0,1.0,2,2,0,2,8.294050
1,3078611,1,1674,2,2.0,1.0,1.0,1.0,1.0,1,...,2,2,5.0,1.0,1.0,1,4,1,3,8.665613
2,3087621,1,1522,2,4.0,1.0,NaN,1.0,1.0,1,...,1,2,1.0,1.0,1.0,2,1,0,1,8.070906
3,3082781,1,1320,2,3.0,1.0,NaN,1.0,1.0,1,...,2,2,1.0,4.0,1.0,1,2,0,2,9.210340
4,3119371,1,1281,2,4.0,1.0,1.0,1.0,1.0,1,...,2,2,1.0,1.0,1.0,1,4,0,4,9.305651


In [168]:
df = df.drop(['REGION', 'PONDERA'], axis = 1)
df.dropna(axis = 1)
df.head()

,id,HomeType,RoomsNumber,FloorMaterial,RoofMaterial,RoofCoat,Water,WaterType,Toilet,ToiletLocation,...,Sink,Garage,Ownership,CookingCombustible,BathroomUse,Working,HouseMembers,Memberless10,Membermore10,TotalHouseHoldIncome
0,3024681,2,2.0,1.0,NaN,1.0,1.0,1,1,1.0,...,2,2,3.0,1.0,1.0,2,2,0,2,8.294050
1,3078611,2,2.0,1.0,1.0,1.0,1.0,1,1,1.0,...,2,2,5.0,1.0,1.0,1,4,1,3,8.665613
2,3087621,2,4.0,1.0,NaN,1.0,1.0,1,1,1.0,...,1,2,1.0,1.0,1.0,2,1,0,1,8.070906
3,3082781,2,3.0,1.0,NaN,1.0,1.0,1,1,1.0,...,2,2,1.0,4.0,1.0,1,2,0,2,9.210340
4,3119371,2,4.0,1.0,1.0,1.0,1.0,1,1,1.0,...,2,2,1.0,1.0,1.0,1,4,0,4,9.305651


In [169]:
df1 = df.iloc[:,1:]

df1.TotalHouseHoldIncome

0       8.294050
1       8.665613
2       8.070906
3       9.210340
4       9.305651
5       8.853665
6       9.380083
7       9.086476
8       7.740664
9       8.809863
10           NaN
11      7.495542
12      7.570443
13      8.101678
14      9.259131
15      9.197255
16      8.961879
17      8.268732
18      9.210340
19      6.802395
20      7.313220
21      9.543235
22      8.699515
23      8.987197
24      9.472705
25      9.190138
26      8.699515
27      9.871739
28      7.807917
29      9.220291
          ...   
2673    7.718685
2674    7.431300
2675    8.176110
2676    6.109248
2677    6.768493
2678    7.313220
2679    7.783224
2680    8.853665
2681    8.575462
2682    7.090077
2683    8.366370
2684    7.796469
2685    7.495542
2686    8.556414
2687    7.649693
2688    7.244228
2689    7.649693
2690    6.802395
2691    7.955074
2692    8.476371
2693    7.463937
2694    8.131531
2695    7.313220
2696    8.691146
2697    8.421563
2698    7.819234
2699    8.476371
2700    8.2335

In [171]:
lm = slm.ols(formula = 'TotalHouseHoldIncome ~ ' + ' + '.join(df1.columns[:-1]), data = df1, missing = 'drop').fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     TotalHouseHoldIncome   R-squared:                       0.359
Model:                              OLS   Adj. R-squared:                  0.352
Method:                   Least Squares   F-statistic:                     47.04
Date:                  Wed, 30 Nov 2016   Prob (F-statistic):          4.14e-189
Time:                          12:24:33   Log-Likelihood:                -2138.0
No. Observations:                  2209   AIC:                             4330.
Df Residuals:                      2182   BIC:                             4484.
Df Model:                            26                                         
Covariance Type:              nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept              4.4606      0.167     26.698      0.000         4.133     4.788
HomeType               0.0638      0.030      2.125      0.034         0.005     0.123
RoomsNumber            0.0237      0.068      0.348      0.728        -0.110     0.157
FloorMaterial         -0.1786      0.046     -3.914      0.000        -0.268    -0.089
RoofMaterial          -0.0158      0.012     -1.274      0.203        -0.040     0.009
RoofCoat              -0.0470      0.044     -1.080      0.280        -0.132     0.038
Water                  0.0042      0.068      0.061      0.951        -0.130     0.138
WaterType             -0.0427      0.037     -1.161      0.246        -0.115     0.029
Toilet                 4.4606      0.167     26.698      0.000         4.133     4.788
ToiletLocation        -0.0893      0.076     -1.183      0.237        -0.237     0.059
ToiletType            -0.1175      0.058     -2.028      0.043        -0.231    -0.004
Sewer                 -0.0694      0.025     -2.784      0.005        -0.118    -0.021
DumpSites              0.0206      0.050      0.409      0.682        -0.078     0.120
Flooding               0.0983      0.039      2.493      0.013         0.021     0.176
EmergencyLoc           0.1317      0.132      1.000      0.317        -0.127     0.390
UsableTotalRooms       0.0783      0.069      1.133      0.257        -0.057     0.214
SleepingRooms         -0.0226      0.028     -0.807      0.420        -0.078     0.032
OfficeRooms           -0.0211      0.049     -0.433      0.665        -0.116     0.074
OnlyWork               0.0598      0.029      2.075      0.038         0.003     0.116
Kitchen                0.1602      0.054      2.943      0.003         0.053     0.267
Sink                  -0.1097      0.030     -3.664      0.000        -0.168    -0.051
Garage                -0.0465      0.032     -1.455      0.146        -0.109     0.016
Ownership             -0.0179      0.009     -1.989      0.047        -0.036    -0.000
CookingCombustible    -0.2108      0.038     -5.526      0.000        -0.286    -0.136
BathroomUse           -0.0325      0.050     -0.654      0.513        -0.130     0.065
Working               -0.7175      0.040    -17.751      0.000        -0.797    -0.638
HouseMembers           0.0408      0.007      5.471      0.000         0.026     0.055
Memberless10          -0.0468      0.013     -3.620      0.000        -0.072    -0.021
Membermore10           0.0876      0.011      8.101      0.000         0.066     0.109
==============================================================================
Omnibus:                      163.549   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              282.213
Skew:                          -0.542   

In [180]:
from sklearn.model_selection import train_test_split

Y = df1.TotalHouseHoldIncome
X = df1.iloc[:,:-1]
#X = X.dropna()
#Y = Y.dropna()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.4, random_state = 200)

,HomeType,RoomsNumber,FloorMaterial,RoofMaterial,RoofCoat,Water,WaterType,Toilet,ToiletLocation,ToiletType,...,Kitchen,Sink,Garage,Ownership,CookingCombustible,BathroomUse,Working,HouseMembers,Memberless10,Membermore10
1,2,2.0,1.0,1.0,1.0,1.0,1,1,1.0,1.0,...,1,2,2,5.0,1.0,1.0,1,4,1,3
4,2,4.0,1.0,1.0,1.0,1.0,1,1,1.0,1.0,...,1,2,2,1.0,1.0,1.0,1,4,0,4
5,2,2.0,1.0,1.0,1.0,1.0,1,1,1.0,1.0,...,1,2,2,5.0,1.0,1.0,1,4,0,4
6,2,4.0,1.0,2.0,1.0,1.0,1,1,1.0,1.0,...,1,2,2,1.0,1.0,1.0,1,4,0,4
15,2,1.0,1.0,1.0,1.0,1.0,1,1,1.0,1.0,...,1,1,2,5.0,1.0,1.0,1,3,0,3
27,1,5.0,1.0,2.0,1.0,1.0,1,1,1.0,1.0,...,1,1,1,1.0,1.0,1.0,1,4,0,4
29,2,2.0,1.0,2.0,1.0,1.0,1,1,1.0,1.0,...,1,1,2,5.0,1.0,1.0,1,3,1,2
30,2,3.0,1.0,1.0,1.0,1.0,1,1,1.0,1.0,...,1,2,2,5.0,1.0,1.0,1,4,0,4
31,2,3.0,1.0,2.0,1.0,1.0,1,1,1.0,1.0,...,1,1,1,1.0,1.0,1.0,1,1,0,1
44,1,3.0,1.0,2.0,1.0,1.0,1,1,1.0,1.0,...,1,1,2,1.0,1.0,1.0,1,3,0,3


In [176]:
lm = linear_model.LinearRegression(fit_intercept = False)
lm.fit(X_train, y_train)
y_hat_IS = lm.predict(X_train)
err_lm_IS = y_hat_IS - y_train
R2lm_IS = 1 - (np.var(err_lm_IS) / np.var(y_train))

y_hat_OS = lm.predict(X_test)
err_lm_OS = y_hat_OS - y_test
R2lm_OS = 1 - (np.var(err_lm_OS) / np.var(y_test))

print('The performances for IS and OS are: {0} and {1}'.format(R2lm_IS, R2lm_OS))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [45]:
PV = []
feats = df1.columns[:-1]
for feat in feats:
    if lm.pvalues[str(feat)] < 0.05:
        PV.append(feat)
df1 = pd.concat([df1['TotalHouseHoldIncome'],df1[PV]],axis = 1)
df1.head()

,TotalHouseHoldIncome,HomeType,Sewer,Flooding,OfficeRooms,OnlyWork,Kitchen,Sink,Garage,CookingCombustible,Working,Memberless10,Membermore10
0,4.0,2,1.0,2.0,2,0,1,2,2,1.0,2,0,2
1,5.8,2,1.0,2.0,2,0,1,2,2,1.0,1,1,3
2,3.2,2,1.0,1.0,2,0,1,1,2,1.0,2,0,1
3,10.0,2,1.0,2.0,2,0,1,2,2,4.0,1,0,2
4,11.0,2,1.0,1.0,2,0,1,2,2,1.0,1,0,4


In [46]:
lm2 = slm.ols(formula = 'TotalHouseHoldIncome ~ ' + ' + '.join(df1.columns[1:]), data = df1).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     TotalHouseHoldIncome   R-squared:                       0.217
Model:                              OLS   Adj. R-squared:                  0.214
Method:                   Least Squares   F-statistic:                     61.31
Date:                  Mon, 21 Nov 2016   Prob (F-statistic):          1.80e-131
Time:                          17:02:14   Log-Likelihood:                -7196.7
No. Observations:                  2662   AIC:                         1.442e+04
Df Residuals:                      2649   BIC:                         1.450e+04
Df Model:                            12                                         
Covariance Type:              nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept              7.1426      0.840      8.506      0.000         5.496     8.789
HomeType               0.4410      0.145      3.050      0.002         0.157     0.725
Sewer                 -0.7449      0.123     -6.045      0.000        -0.987    -0.503
Flooding               0.8700      0.207      4.207      0.000         0.464     1.275
OfficeRooms            0.4045      0.256      1.583      0.114        -0.097     0.906
OnlyWork               0.4836      0.148      3.263      0.001         0.193     0.774
Kitchen                0.6691      0.286      2.342      0.019         0.109     1.229
Sink                  -0.7970      0.149     -5.339      0.000        -1.090    -0.504
Garage                -0.4947      0.169     -2.924      0.003        -0.826    -0.163
CookingCombustible    -1.2507      0.180     -6.967      0.000        -1.603    -0.899
Working               -2.6103      0.199    -13.144      0.000        -3.000    -2.221
Memberless10          -0.3163      0.094     -3.347      0.001        -0.502    -0.131
Membermore10           0.6716      0.056     11.918      0.000         0.561     0.782
==============================================================================
Omnibus:                     2158.237   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           108203.895
Skew:                           3.456   Prob(JB):                         0.00
Kurtosis:                      33.459   Cond. No.                         67.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

REGION                  NaN
PONDERA                 1.0
HomeType                1.0
RoomsNumber             1.0
FloorMaterial           1.0
RoofMaterial            1.0
RoofCoat                1.0
Water                   1.0
WaterType               1.0
Toilet                  1.0
ToiletLocation          1.0
ToiletType              1.0
Sewer                   1.0
DumpSites               1.0
Flooding                1.0
EmergencyLoc            1.0
UsableTotalRooms        1.0
SleepingRooms           1.0
OfficeRooms             1.0
OnlyWork                1.0
Kitchen                 1.0
Sink                    1.0
Garage                  1.0
Ownership               1.0
CookingCombustible      1.0
BathroomUse             1.0
Working                 1.0
HouseMembers            1.0
Memberless10            1.0
Membermore10            1.0
TotalHouseHoldIncome    1.0
dtype: float64